In [1]:
import openai
import json
from pydantic import BaseModel, Field
from typing import List,Optional
from langchain.tools import tool
from langchain.agents import Tool
import time
from tqdm import tqdm
openai.api_key = ""
GPT_MODEL = "gpt-3.5-turbo-1106"

In [2]:
def get_completion_from_messages(messages, model=GPT_MODEL, temperature=0, max_tokens=1000):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response

In [3]:
json_schema = {
  "type": "array",
  "items": {
      "type": "object",
      "properties": {
        "tool_name": { "type": "string" },
        "arguments": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "argument_name": { "type": "string" },
              "argument_value": { "type": "string" }
            },
          "required": ["argument_name", "argument_value"]
          }
        }
      },
  "required": ["tool_name", "arguments"]
  }
}

In [4]:
json_schema_2 = """[
    "tool_name":"function_name",
    "arguments":[
        {
            "argument_name":"",
            "argument_value":""
        }
    ],
    },
]"""

In [5]:
def clean_output(strin):
    ok = strin.replace("`","")
    ok = ok.replace("json","")
    return ok

In [6]:
delimiter_2 = "$$$$"
system_message_2 = """
An example that you can refer to is:

<user prompt>:"Summarize high severity tickets from the customer UltimateCustomer"

Response to user:<delimiter>
"tool_name": "search_object_by_name",
"arguments": [
    {
        "argument_name": "query",
        "argument_value": "UltimateCustomer"
    }
]
},
{
"tool_name": "works_list",
"arguments": [
    {
        "argument_name": "ticket.rev_org",
        "argument_value": ["$$PREV[0]"]
    },
    {
        "argument_name": "ticket.severity",
        "argument_value": ["high"]
    },
    {
        "argument_name": "type",
        "argument_value": ["ticket"]
    }
]
},
{
"tool_name": "summarize_objects",
"arguments": [
    {
        "argument_name": "objects",
        "argument_value": "$$PREV[1]"
    }
]
}
"""

In [7]:
tools = """
def summarize_objects(objects: list[object]) -> str:
    ""
    Summarizes a list of objects. The logic of how to summarize a
    particular object type is an internal implementation detail.

    Args:
        objects (List[object]): List of objects to summarize.

    Returns:
        str: A summarized string describing the objects.
    ""

def prioritize_objects(objects: list[object]) -> list[object]:
    ""
    Returns a list of objects sorted by priority. The logic of what constitutes
    priority for a given object is an internal implementation detail.

    Args:
        objects (list[object]): A list of objects to be prioritized.

    Returns:
        list[object]: A list of objects sorted by priority.
    ""

def get_similar_work_items(work_id: str) -> list[str]:
    ""
    Returns a list of work items that are similar to the given work item.

    Args:
        work_id (str): The ID of the work item for which you want to find similar items.

    Returns:
        list[str]: A list of work item IDs that are similar to the given work item.
    ""

def add_work_items_to_sprint(work_ids: list[str], sprint_id: str) -> None:
    ""
    Adds the given work items to the sprint.

    Args:
        work_ids (list[str]): A list of work item IDs to be added to the sprint.
        sprint_id (str): The ID of the sprint to which the work items should be added.
    ""

def get_sprint_id() -> str:
    ""
    Returns the ID of the current sprint.
    ""
    
def create_actionable_tasks_from_text(text: str) -> list[str]:
    ""
    Given a text, extracts actionable insights and creates tasks for them, which are a kind of work item.

    Args:
        text (str): The text from which the actionable insights need to be created.

    Returns:
        list[str]: A list of actionable tasks created from the text.
    ""
    
def who_am_i() -> str:
    ""
    Returns the ID of the current user.
    ""
    
def search_object_by_name(query: str) -> str:
    ""
    Given a search string, returns the ID of a matching object in the system of record.
    If multiple matches are found, it returns the one where the confidence is highest.

    Args:
        query (str): The search string, could be, for example, a customer's name, part name, or user name.

    Returns:
        str: The ID of the matching object with the highest confidence.
    ""
def works_list(
    applies_to_part: list[str],  created_by: list[str],  issue_priority: list[str],  issue_rev_orgs: list[str], limit: 50,  owned_by: list[str],
    stage_name: list[str], ticket_needs_response: bool, ticket_rev_org: list[str], ticket_severity: list[str], ticket_source_channel: list[str],
    work_type: list[str]) -> list[object]:
    ""
    Returns a list of work items matching the request.

    Returns:
        List[object]: A list of work items matching the request.
    ""
"""

In [8]:
delimiter = "#####"
system_message = f"""
You will be provided with function calling queries.The queries will be delimited with \
{delimiter} characters. \
Think step by step to answer the queries.\

Step 1:{delimiter}First, decide whether the query is relevant to the tools or not.If not output an empty list. \

Step 2:{delimiter}Second, decide all the functions that the user is asking us to use.

Step 3:{delimiter}Third,decide the sequence in which the user wants the functions to be called,and the output of which tool is required in\
the chain of tools and to reference the returned value upon calling of the ith tool in the chain, use $$PREV[i] as argument value where. \ 
i = 0, 1, .. j-1; where j is the number of tools used.\

Step 4:{delimiter}Fourth,check if the flow of the query is kept in the sequence of tools.

Provide your output in the JSON format given in {json_schema_2}.\

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to add {delimiter} to seperate every step

The functions are given below:
{tools}        
"""

user_message_1 = """
Summarize high severity tickets
from the customer
UltimateCustomer
"""

user_message_2 = "What is the meaning of life??"
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'system', 
 'content': system_message_2},
{'role':'user',
'content': f"{delimiter}{user_message_1}{delimiter}"}
] 
response = get_completion_from_messages(messages)
print(response)

ChatCompletion(id='chatcmpl-8UzDZrss8G5Y7VHgDQqH55iJhf5mz', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Step 1:#####\nThe query is relevant to the tools.\n\nStep 2:#####\nThe user is asking us to use the following functions:\n- search_object_by_name\n- works_list\n- summarize_objects\n\nStep 3:#####\nThe sequence in which the user wants the functions to be called is as follows:\n1. search_object_by_name\n2. works_list\n3. summarize_objects\n\nThe output of the second tool is required in the chain of tools.\n\nStep 4:#####\nThe flow of the query is kept in the sequence of tools.\n\nResponse to user:#####\n[\n    {\n        "tool_name": "search_object_by_name",\n        "arguments": [\n            {\n                "argument_name": "query",\n                "argument_value": "UltimateCustomer"\n            }\n        ]\n    },\n    {\n        "tool_name": "works_list",\n        "arguments": [\n            {\n                "argument_name": "tic

In [9]:
print(response.choices[0].message.content)

Step 1:#####
The query is relevant to the tools.

Step 2:#####
The user is asking us to use the following functions:
- search_object_by_name
- works_list
- summarize_objects

Step 3:#####
The sequence in which the user wants the functions to be called is as follows:
1. search_object_by_name
2. works_list
3. summarize_objects

The output of the second tool is required in the chain of tools.

Step 4:#####
The flow of the query is kept in the sequence of tools.

Response to user:#####
[
    {
        "tool_name": "search_object_by_name",
        "arguments": [
            {
                "argument_name": "query",
                "argument_value": "UltimateCustomer"
            }
        ]
    },
    {
        "tool_name": "works_list",
        "arguments": [
            {
                "argument_name": "ticket_rev_org",
                "argument_value": ["$$PREV[0]"]
            },
            {
                "argument_name": "ticket_severity",
                "argument_value": ["hi

In [10]:
res = clean_output(response.choices[0].message.content)

In [11]:
print(res)

Step 1:#####
The query is relevant to the tools.

Step 2:#####
The user is asking us to use the following functions:
- search_object_by_name
- works_list
- summarize_objects

Step 3:#####
The sequence in which the user wants the functions to be called is as follows:
1. search_object_by_name
2. works_list
3. summarize_objects

The output of the second tool is required in the chain of tools.

Step 4:#####
The flow of the query is kept in the sequence of tools.

Response to user:#####
[
    {
        "tool_name": "search_object_by_name",
        "arguments": [
            {
                "argument_name": "query",
                "argument_value": "UltimateCustomer"
            }
        ]
    },
    {
        "tool_name": "works_list",
        "arguments": [
            {
                "argument_name": "ticket_rev_org",
                "argument_value": ["$$PREV[0]"]
            },
            {
                "argument_name": "ticket_severity",
                "argument_value": ["hi

In [12]:
final_response = res.split(delimiter)[-1].strip()

In [13]:
print(final_response)

[
    {
        "tool_name": "search_object_by_name",
        "arguments": [
            {
                "argument_name": "query",
                "argument_value": "UltimateCustomer"
            }
        ]
    },
    {
        "tool_name": "works_list",
        "arguments": [
            {
                "argument_name": "ticket_rev_org",
                "argument_value": ["$$PREV[0]"]
            },
            {
                "argument_name": "ticket_severity",
                "argument_value": ["high"]
            },
            {
                "argument_name": "work_type",
                "argument_value": ["ticket"]
            }
        ]
    },
    {
        "tool_name": "summarize_objects",
        "arguments": [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[1]"
            }
        ]
    }
]


In [14]:
def clearning(response):
    ans = response.choices[0].message.content
    ans = clean_output(ans)
    final_ans = ans.split(delimiter)[-1].strip()
    return final_ans

In [15]:
print(clearning(response))

[
    {
        "tool_name": "search_object_by_name",
        "arguments": [
            {
                "argument_name": "query",
                "argument_value": "UltimateCustomer"
            }
        ]
    },
    {
        "tool_name": "works_list",
        "arguments": [
            {
                "argument_name": "ticket_rev_org",
                "argument_value": ["$$PREV[0]"]
            },
            {
                "argument_name": "ticket_severity",
                "argument_value": ["high"]
            },
            {
                "argument_name": "work_type",
                "argument_value": ["ticket"]
            }
        ]
    },
    {
        "tool_name": "summarize_objects",
        "arguments": [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[1]"
            }
        ]
    }
]


In [16]:
to_test = [
    "Summarize work items similar to don:core:dvrv-us-1:devo/0:issue/1",
    "What is the meaning of life?",
    "What is the meaning of life?",
    "Summarize high severity tickets from the customer UltimateCustomer",
    "What are my all issues in the triage stage under part FEAT-123? Summarize them.",
    "List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them.",
    "Given a customer meeting transcript T, create action items and add them to my current sprint.",
    "Get all work items similar to TKT-123, summarize them, create issues from that summary, and prioritize them"
]

In [17]:
def interact(string):
    messages =  messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'system', 
 'content': system_message_2},
{'role':'user',
'content': f"{delimiter}{string}{delimiter}"}
]
    our_ans = get_completion_from_messages(messages)
    print(clearning(our_ans))

In [18]:
start = time.time
for x in tqdm(to_test):
    interact(x)
    print("-------------x-------------x-------------x-------------x-------------x--------------")

 12%|█████▋                                       | 1/8 [00:03<00:24,  3.47s/it]

[
    {
        "tool_name": "get_similar_work_items",
        "arguments": [
            {
                "argument_name": "work_id",
                "argument_value": "don:core:dvrv-us-1:devo/0:issue/1"
            }
        ]
    },
    {
        "tool_name": "summarize_objects",
        "arguments": [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[0]"
            }
        ]
    }
]
-------------x-------------x-------------x-------------x-------------x--------------


 25%|███████████▎                                 | 2/8 [00:04<00:11,  1.84s/it]

[]
-------------x-------------x-------------x-------------x-------------x--------------


 38%|████████████████▉                            | 3/8 [00:07<00:11,  2.35s/it]

...

{
    "tool_name": "search_object_by_name",
    "arguments": [
        {
            "argument_name": "query",
            "argument_value": "Cust123"
        }
    ]
},
{
    "tool_name": "works_list",
    "arguments": [
        {
            "argument_name": "ticket_rev_org",
            "argument_value": ["$$PREV[0]"]
        },
        {
            "argument_name": "ticket_severity",
            "argument_value": ["high"]
        },
        {
            "argument_name": "work_type",
            "argument_value": ["ticket"]
        }
    ]
},
{
    "tool_name": "summarize_objects",
    "arguments": [
        {
            "argument_name": "objects",
            "argument_value": "$$PREV[1]"
        }
    ]
}
-------------x-------------x-------------x-------------x-------------x--------------


 88%|███████████████████████████████████████▍     | 7/8 [00:36<00:06,  6.19s/it]

[
{
"tool_name": "create_actionable_tasks_from_text",
"arguments": [
    {
        "argument_name": "text",
        "argument_value": "T"
    }
]
},
{
"tool_name": "get_sprint_id",
"arguments": []
},
{
"tool_name": "add_work_items_to_sprint",
"arguments": [
    {
        "argument_name": "work_ids",
        "argument_value": ["$$PREV[0]"]
    },
    {
        "argument_name": "sprint_id",
        "argument_value": "$$PREV[1]"
    }
]
}
]
-------------x-------------x-------------x-------------x-------------x--------------


100%|█████████████████████████████████████████████| 8/8 [00:40<00:00,  5.04s/it]

[
    {
        "tool_name": "get_similar_work_items",
        "arguments": [
            {
                "argument_name": "work_id",
                "argument_value": "TKT-123"
            }
        ]
    },
    {
        "tool_name": "summarize_objects",
        "arguments": [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[0]"
            }
        ]
    },
    {
        "tool_name": "prioritize_objects",
        "arguments": [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[1]"
            }
        ]
    }
]
-------------x-------------x-------------x-------------x-------------x--------------
